Нейросеть для определения класса изображения: кошка или собака

In [5]:
"""Подключение библиотек"""

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
## pip install tensorflow-datasets
import tensorflow_datasets as tfds
import tensorflow as tf
import logging
import numpy as np
import time

2024-08-08 22:15:02.279503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 22:15:02.301591: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 22:15:02.308235: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-08 22:15:02.325038: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-08 22:15:03.345147: W tensorflow/compiler/tf2

In [6]:
"""Модель нейросети"""

def dog_cat_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(units=128, activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  model.compile(optimizer=Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy'])
  return model

In [10]:
"""Обучение нейросети.
В исходном датасете — 25 тысяч изображений.
Датасет делится на 3 части: для обучения, верификации и тестирования,
каждое изображение обрабатывается.
Размер изображения приводится к 128 × 128 пикселей, а цвет в диапазоне от нуля до единицы."""

def dog_cat_train(model):
  #splits = tfds.Split.TRAIN.subsplit(weighted=(80, 10, 10))
  splits = ("train[:80%]", "train[19%:]", "train[19%:80%]")
  (cat_train, cat_valid, cat_test), info = tfds.load('cats_vs_dogs',
    split=list(splits), with_info=True, as_supervised=True)

  def pre_process_image(image, label):
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = tf.image.resize(image, (128, 128))
    return image, label

  BATCH_SIZE = 32
  SHUFFLE_BUFFER_SIZE = 1000
  train_batch = cat_train.map(pre_process_image).shuffle(SHUFFLE_BUFFER_SIZE).repeat().batch(BATCH_SIZE)
  validation_batch = cat_valid.map(pre_process_image).repeat().batch(BATCH_SIZE)

  t_start = time.time()
  model.fit(train_batch, steps_per_epoch=4000, epochs=2,
    validation_data=validation_batch,
    validation_steps=10,
    callbacks=None)
  print("Training done, dT:", time.time() - t_start)

In [ ]:
"""обучение модели и сохранение результата"""

model = dog_cat_model()
dog_cat_train(model)
model.save('dogs_cats.keras')

In [31]:
"""Использование обученной сети.
Функция распознавания изображения."""

def dog_cat_predict(model, image_file):
  label_names = ["кошка", "собака"]

  img = keras.preprocessing.image.load_img(image_file,
    target_size=(128, 128))
  img_arr = np.expand_dims(img, axis=0) / 255.0
  #result = model.predict_classes(img_arr)
  predict_x = model.predict(img_arr) 
  result = np.argmax(predict_x,axis=0)
  
  #print("Результат: %s" % label_names[result[0][0]])
  print("Результат: %s" % label_names[result[0]])
  print(f"Результат: {result[0]}")

In [32]:
model = tf.keras.models.load_model('dogs_cats.keras')
dog_cat_predict(model, "cat1.jpeg")
dog_cat_predict(model, "cat2.jpeg")
dog_cat_predict(model, "cat3.jpeg")

dog_cat_predict(model, "dog1.jpeg")
dog_cat_predict(model, "dog2.jpeg")
dog_cat_predict(model, "dog3.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Результат: кошка
Результат: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Результат: кошка
Результат: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Результат: кошка
Результат: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Результат: кошка
Результат: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Результат: кошка
Результат: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Результат: кошка
Результат: 0


В результате всех экспериментов, нейросеть все равно считает, что любое переданное ей изображение в день кошки 8 августа должно быть кошкой. Поэтому собаку сегодня тоже считает кошкой.
